In [1]:
import logging, os
logging.disable(logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import geopandas as gpd
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import datetime
from pathlib import Path

import plotly.express as px

import tensorflow as tf

from tensorflow.keras.layers import *
from tensorflow.keras.initializers import *

from sklearn.model_selection import train_test_split


import keras_tuner as kt

from tqdm.notebook import tqdm

from importlib import reload
import sentinel_utils
import plot_utils
from data_generator import DataGenerator

In [2]:
def get_metrics():
    prc = tf.keras.metrics.AUC(name='prc', curve='PR')

    f1_scores = []
    for average in ['micro', 'macro', 'weighted']:
        f1_scores.append(
            tf.keras.metrics.F1Score(
                average=average, threshold=0.5, name=f'{average}f1score')
        )
    metrics = [
        'accuracy', 'recall', 'precision', 'auc', prc
    ] + f1_scores

    return metrics

In [3]:
class BuildHyperModel(kt.HyperModel):
    def __init__(self, **kwargs):
        vars(self).update(kwargs)
        self.kwargs = kwargs
        
    def build(self, hp):
        sentinel_10m_input = Input((100, 100, 2))
        sentinel_20m_input = Input((50, 50, 2))
        
        x = concatenate([sentinel_10m_input, UpSampling2D(2)(sentinel_20m_input)])
        filter_power = hp.Int(
            'filters_power', min_value=3, max_value=7, step=1
        )
        for filters_scale in [2**x for x in range(filter_power+1)]:
            x = Conv2D(
                filters=filters_scale,
                kernel_size=hp.Int('kernel_size', min_value=3, max_value=7, step=2),
                padding='same',
                activation='relu',
            )(x)
            x = MaxPooling2D(pool_size=2, strides=2, padding='same')(x)    
            x = BatchNormalization()(x)
        
        x = Flatten()(x)

        units_power = hp.Int(
            'units_power', min_value=4, max_value=8, step=1
        )
        for units_scale in reversed([2**x for x in range(filter_power+1)]):
            x = Dense(units_scale, activation='relu')(x)
            x = BatchNormalization()(x)

        x = Dropout(hp.Float('dropout_rate', min_value=0.0, max_value=0.8, step=0.1))(x)

        if self.output_bias:
            output_bias = tf.keras.initializers.Constant(self.output_bias)
        else:
            output_bias = None

        outputs = Dense(
            self.output_shape,
            bias_initializer=output_bias,
            activation='sigmoid',   
        )(x)

        m = tf.keras.models.Model(
            inputs=[sentinel_10m_input, sentinel_20m_input], 
            outputs=outputs
        )

        m.compile(
            optimizer=hp.Choice('optimizer', ['adam', 'sgd']), 
            loss=self.loss, metrics=self.metrics)
        
        return m

In [4]:
reload(sentinel_utils)

model_dir = Path('models', 'selected_model')
shards_dir = Path.home().joinpath('sentinel_data', 'shards')

utils = sentinel_utils.SentinelUtils(min_occurrences=20000)

selected_classes = utils.get_processed_labels()

data_summary = utils.get_data_summary(
    shards_dir, selected_classes, overwrite_existing=False
)

loss = 'binary_crossentropy'

params = dict(
    loss=loss,
    shards_dir=shards_dir,
    selected_classes=selected_classes,
    data_summary=data_summary,
    batch_size=32
)

In [ ]:
training_ids, test_ids = train_test_split(
    selected_classes.index, test_size=10000, random_state=42
)

training_generator = DataGenerator(training_ids, shuffle=True, **params)
testing_generator = DataGenerator(test_ids, shuffle=False, **params)

metrics = get_metrics()

hypermodel = BuildHyperModel(**dict(
    output_shape=selected_classes.shape[1], 
    metrics=metrics, 
    loss=loss,
    output_bias=data_summary['initial_bias']
))

tuner = kt.Hyperband(
    hypermodel,
    objective=kt.Objective('val_recall', direction='max'),
    directory=Path('trials'),
    project_name='hyperband',
    overwrite=True
)
     
tuner.search(
    x=training_generator,
    validation_data=testing_generator,
    class_weight=data_summary['class_weights'],
    verbose=1
)

Trial 5 Complete [00h 04m 42s]
val_recall: 0.4115864336490631

Best val_recall So Far: 0.4383960962295532
Total elapsed time: 00h 23m 32s

Search: Running Trial #6

Value             |Best Value So Far |Hyperparameter
3                 |7                 |filters_scale
7                 |7                 |kernel_size
5                 |8                 |units_power
0.1               |0.1               |dropout_rate
sgd               |adam              |optimizer
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
4                 |4                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
7100/7100 ━━━━━━━━━━━━━━━━━━━━ 142s 19ms/step - accuracy: 0.2821 - auc: 0.6990 - loss: 0.5209 - macrof1score: 0.0729 - microf1score: 0.1562 - prc: 0.4373 - precision: 0.3627 - recall: 0.0386 - weightedf1score: 0.1318 - val_accuracy: 0.2743 - val_auc: 0.7121 - val_loss: 0.6201 - val_macrof1score: 0.1675 - val

In [ ]:
reload(keras_model_creator)
model, testing_generator = keras_model_creator.KerasModelCreator(**params).run()

In [ ]:
# model.summary()